In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests

In [2]:
directory = directory_scratch
exp = get_exp((1,0,35),directory)

begin = 2021-03-15 15:31:00 
  end = 2021-03-18 13:31:00


In [4]:
exp.load_compressed_skel()

In [7]:
RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(get_exp((40,15,15+37),directory),criter)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


/home/cbisot/pycode/MscThesis/sample/notebooks/analysis/util.py:79: RuntimeWarning: invalid value encountered in double_scalars
  return(np.sum(weighted)/np.sum(lengths))


In [107]:
def get_orientation(hypha,t,start,length=50):
    nodes, edges = hypha.get_nodes_within(t)
    pixel_list_list = []
#     print(edges[start:])
    for edge in edges[start:]:
        pixel_list_list+=edge.pixel_list(t)
    pixel_list = np.array(pixel_list_list)
    vector = pixel_list[min(length, len(pixel_list) - 1)] - pixel_list[0]
    unit_vector = vector / np.linalg.norm(vector)
    vertical_vector = np.array([-1, 0])
    dot_product = np.dot(vertical_vector, unit_vector)
    if (
        vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
    ):  # determinant
        angle = np.arccos(dot_product) / (2 * np.pi) * 360
    else:
        angle = -np.arccos(dot_product) / (2 * np.pi) * 360
    return angle

In [108]:
get_orientation(rh,t,0),get_orientation(rh,t,1),get_orientation(rh,t,2)

(61.699244233993625, 23.355564859286005, 50.9061411137705)

In [127]:
plt.close('all')
branch_root = []
branch_anastomose = []
two_time = []
angles = []
for rh in RH:
#     rh = choice(RH)
    t = rh.ts[-1]
    nodes, edges = rh.get_nodes_within(t)
    for i,node in enumerate(nodes[1:-1]):
        found = False
        for hyph in exp.hyphaes:
            if hyph.root.label == node:
                if found:
                    two_time.append(hyph.root)
                branch_root.append(hyph.root)
                if t in hyph.ts:
                    nodes_h,edges_h = hyph.get_nodes_within(t)
                    if len(edges_h)>0:
                        edge_main = edges[i+1]
                        edge_branch = edges_h[0]
                        angle_main = get_orientation(rh,t,i+1,100)
                        angle_branch = get_orientation(hyph,t,0,100)
                        angles.append(((angle_main-angle_branch),(rh,hyph,t)))
#                         print(node,edges[i+1],edges_h[0],angle_main-angle_branch)
#                         exp.plot([t],[[node,edge_main.begin.label,edge_main.end.label,edge_branch.begin.label,edge_branch.end.label]])
                        found = True
        if not found:
            branch_anastomose.append(Node(node,exp))
#     break


In [128]:
angles_180 = [(angle+180)%360-180  for angle,infos in angles]

In [129]:
from scipy import stats
result = stats.ttest_1samp(angles_180, 0)
result.pvalue

3.911535633479534e-06

In [130]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(angles_180,30,density = False)
ax.set_xlabel('branching angle (°)')
ax.set_ylabel('Density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Density')

In [139]:
angles_rh = [c[0] for c in angles if c[1][1] in RH]
angles_bas = [c[0] for c in angles if c[1][1] in BAS]
fig = plt.figure()
bins = np.linspace(-180, 180, 30)
ax = fig.add_subplot(111)
ax.hist(angles_rh,bins,alpha=0.5,label='rh',density=True)
ax.hist(angles_bas,bins,alpha=0.5,label='bas',density=True)
ax.set_xlabel(r'branching angle ($°$)')
plt.legend(loc='upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
fig = plt.figure()
bins = np.linspace(-180, 180, 30)
ax = fig.add_subplot(111)
ax.hist(np.abs(angles_rh),bins,alpha=0.5,label='rh',density=True)
ax.hist(np.abs(angles_bas),bins,alpha=0.5,label='bas',density=True)
ax.set_xlabel(r'branching angle ($°$)')
plt.legend(loc='upper right')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [146]:
example_rh = [c[1] for c in angles if c[1][1] in RH and c[0]>90]

In [149]:
len(example_rh)

7

In [141]:
np.mean(np.abs(angles_rh)),np.mean(np.abs(angles_bas))

(69.03228043117815, 105.0778356812492)

In [176]:
angle0 = [((c[0]+180)%360-180,c[1]) for c in angles if abs((c[0]+180)%360-180)>120]

In [190]:
len(angle0)

92

In [191]:
len(angles)

730

In [206]:
angle,(rh,hyph,t) = choice(angle0)
angle

-136.22079868683977

In [208]:
plt.close('all')
exp.plot([t],[[rh.root.label,rh.end.label,hyph.root.label,hyph.end.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [209]:
rh,hyph

(Hyphae(14339,12307), Hyphae(16968,15559))

In [207]:
hyph.end.show_source_image(t,t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
edge_branch.width(t)

46.190082644628106

In [92]:
rh,nodes,rh.root,edges

(Hyphae(17512,17501),
 [17501, 19006, 19016, 19030, 17512],
 Node(17501),
 [Edge(17501,19006), Edge(19006,19016), Edge(19016,19030), Edge(19030,17512)])

In [14]:
len(branch_root),len(branch_anastomose),len(two_time),len(RH)

(1093, 493, 327, 180)

In [88]:
node = choice(branch_anastomose)
node.ts()

[6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [89]:
plt.close('all')
node.show_source_image(node.ts()[-1],node.ts()[-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
len(branch_root), len(branch_anastomose)

(11063, 64776677)